# Área Compartilhada

Bloco 1: Instalação e Imports
Esta é a primeira célula do seu notebook.

In [11]:
# Célula 1: Instalações e Imports
# Instalações (descomente se necessário)
# !pip install -q google-generativeai pandas python-dotenv

import google.generativeai as genai
import pandas as pd
import pathlib
import json
import numpy as np
import time
import os
from dotenv import load_dotenv

# Carrega as variáveis de ambiente do arquivo .env
load_dotenv()

True

Bloco 2: Configuração de API Key
Configure sua API Key do Gemini no arquivo .env (copie .env.example para .env e preencha)

In [12]:
# Célula 2: Configuração da API Key do Gemini

# Obter a API Key do arquivo .env
api_key = os.getenv('GEMINI_API_KEY')

if not api_key:
    print("ERRO: API Key não encontrada!")
    print("Por favor:")
    print("1. Copie o arquivo .env.example para .env")
    print("2. Edite o arquivo .env e adicione sua API Key do Gemini")
    print("3. Obtenha sua API Key em: https://aistudio.google.com/apikey")
else:
    genai.configure(api_key=api_key)
    print("API Key configurada com sucesso!")

API Key configurada com sucesso!


# INDEXAÇÃO (Gerar Legendas e Embeddings)

Objetivo: Rodar uma única vez sobre sua pasta de fotos. Ele vai:

Ler todas as fotos da pasta informada.

Para cada foto, gerar a legenda (alt-text).

Para cada foto, gerar o embedding (vetor de busca).

Salvar tudo em um único arquivo index.csv na mesma pasta.

Bloco 3: Configuração de Pastas
Digite a pasta onde estão suas fotos quando solicitado.

In [13]:
# Célula 3: Configuração de Pastas

# 1. Solicitar ao usuário a pasta das fotos
# Exemplo Windows: C:/Users/SeuNome/Pictures/Minhas_Fotos
# Exemplo Linux/Mac: /home/usuario/fotos
PASTA_DAS_FOTOS = input("Digite o caminho da pasta com suas fotos: ").strip()

# Remover aspas do caminho (simples ou duplas) que podem vir ao copiar do Windows
PASTA_DAS_FOTOS = PASTA_DAS_FOTOS.strip('"').strip("'")

# Validar se a pasta existe
if not os.path.exists(PASTA_DAS_FOTOS):
    print(f"ERRO: A pasta '{PASTA_DAS_FOTOS}' não existe!")
    print("Por favor, verifique o caminho e tente novamente.")
else:
    print(f"✓ Pasta encontrada: {PASTA_DAS_FOTOS}")
    
    # 2. Definir o arquivo de saída automaticamente como index.csv na mesma pasta
    ARQUIVO_DE_SAIDA = os.path.join(PASTA_DAS_FOTOS, "index.csv")
    print(f"✓ O índice será salvo em: {ARQUIVO_DE_SAIDA}")

    # 3. Modelos de IA
    MODELO_VISAO = 'models/gemini-2.0-flash'      # Para gerar legendas
    MODELO_EMBEDDING = 'models/embedding-001'     # Para gerar vetores de busca (DE TEXTO)

    # 4. Prompt para a legenda (COM CHAVES "ESCAPADAS" {{ }})
    PROMPT_LEGENDA = """
Sua única tarefa é analisar a imagem e retornar um objeto JSON.

VOCÊ É UM FOTÓGRAFO PROFISSIONAL E ESPECIALISTA EM CATALOGAÇÃO DE PORTFÓLIO.
Seu objetivo é catalogar esta imagem para um portfólio de fotografia profissional, 
identificando pessoas específicas quando possível para facilitar buscas futuras.

---
INFORMAÇÕES DE CONTEXTO DA SESSÃO:
{session_context}
---

INSTRUÇÕES CRÍTICAS PARA IDENTIFICAÇÃO DE PESSOAS:

1. PRIORIDADE MÁXIMA: Se a pessoa na foto corresponde às características da CLIENTE PRINCIPAL 
   (marcada como "PRIMARY" no contexto), USE O NOME DELA na legenda, contexto e keywords.
   
2. Para pessoas marcadas como "SECONDARY" ou "TERTIARY", use seus nomes quando tiver 
   confiança razoável na identificação baseada nas pistas visuais fornecidas.
   
3. Use termos genéricos (mãe, filho, mulher, homem, criança) APENAS quando não puder 
   identificar com certeza quem é a pessoa.
   
4. CONSISTÊNCIA: Mantenha o mesmo padrão de identificação em fotos similares. Se identificou 
   alguém pelo nome, continue usando esse nome em outras fotos onde a pessoa apareça.
   
5. Para acessibilidade, combine descrição visual + nome específico quando possível.
   Exemplo: "Juliana Lombardi sorrindo com sua filha" ao invés de apenas "mãe e filha".

EXEMPLOS DE BOA CATALOGAÇÃO:
- ✓ "Juliana Lombardi e sua filha brincando no sofá"
- ✓ "Retrato de Juliana Lombardi sorrindo, luz natural"
- ✓ "Família reunida: Juliana, marido e filha no apartamento"
- ✗ "Mãe e filha brincando" (muito genérico quando se sabe quem é)
- ✗ "Mulher sorrindo" (muito genérico quando se sabe quem é)

ESTRUTURA DO JSON (OBRIGATÓRIA):
{{
  "caption": "Legenda descritiva incluindo NOMES das pessoas identificadas. Foque no sujeito, ação e cenário.",
  "keywords": {{
    "subjects": ["Nome Pessoa 1", "Nome Pessoa 2", "objeto1", "local"],
    "techniques": ["técnica_fotográfica", "tipo_de_luz"],
    "mood": ["emoção", "atmosfera"]
  }},
  "category": "retrato | familia | grupo | produto | evento | paisagem",
  "context": "Descrição detalhada do contexto da cena incluindo NOMES das pessoas identificadas. Use as informações de contexto da sessão para enriquecer esta descrição."
}}

IMPORTANTE: 
- Sempre use nomes próprios quando disponíveis no contexto
- Keywords de "subjects" devem priorizar nomes de pessoas sobre termos genéricos
- Responda APENAS com o JSON, sem texto adicional
- Sempre responda em português BR
"""

✓ Pasta encontrada: C:\Users\Leandro.Cherubini\GitHub\photoIndexer\.test
✓ O índice será salvo em: C:\Users\Leandro.Cherubini\GitHub\photoIndexer\.test\index.csv


Bloco 4: O Script Principal (Indexador)
Este é o bloco mais importante. Ele vai demorar para rodar se você tiver muitas fotos (pois faz 2 chamadas de API por foto e espera 1 segundo).

In [ ]:
# Célula 4: Script de Indexação (Versão 8.0 - Corrigido duplicação de arquivos)

import json # Certifique-se de que o json está importado
import os # Importar os para verificação de arquivo

# 1. Carregar os modelos
model_visao = genai.GenerativeModel(MODELO_VISAO)
model_embedding = genai.GenerativeModel(MODELO_EMBEDDING)

# 2. Encontrar as fotos (com deduplicação para evitar processar a mesma foto 2x)
pasta = pathlib.Path(PASTA_DAS_FOTOS)

# Buscar todos os arquivos com extensões variadas
extensoes = ['*.jpg', '*.jpeg', '*.png', '*.JPG', '*.JPEG', '*.PNG']
arquivos_encontrados = []
for ext in extensoes:
    arquivos_encontrados.extend(pasta.glob(ext))

# CORREÇÃO: Deduplica usando o caminho absoluto em lowercase (Windows = case-insensitive)
# Isso evita processar IMG_8000.jpg e IMG_8000.JPG como arquivos diferentes
arquivos_unicos = {}
for arquivo in arquivos_encontrados:
    # Usa o caminho completo em lowercase como chave única
    chave = str(arquivo.absolute()).lower()
    if chave not in arquivos_unicos:
        arquivos_unicos[chave] = arquivo

arquivos_fotos = list(arquivos_unicos.values())

print(f"Encontradas {len(arquivos_fotos)} fotos únicas. Iniciando indexação...")

# --- OTIMIZAÇÃO: Ler o session.json UMA VEZ, fora do loop ---
session_json_content = "Nenhum contexto de sessão fornecido." # Padrão
caminho_sessao = pasta / "session.json" # Procura o JSON na pasta principal

if caminho_sessao.exists():
    try:
        with open(caminho_sessao, 'r', encoding='utf-8') as f:
            session_json_content = f.read()
        print(f"Contexto 'session.json' carregado com sucesso.")
    except Exception as e:
        print(f"--- Aviso: Falha ao ler o session.json. Erro: {e}")
else:
    print("--- Aviso: 'session.json' não encontrado na pasta. Usando prompt padrão.")

# Formatar o prompt final UMA VEZ, com o contexto da sessão
final_prompt = PROMPT_LEGENDA.format(session_context=session_json_content)
# --- FIM DA OTIMIZAÇÃO ---


# 3. Loop de processamento (Agora muito mais limpo)
resultados_finais = []
for caminho_foto in arquivos_fotos:
    arquivo_upload = None  # Inicia a variável como nula
    try:
        print(f"Processando: {caminho_foto.name}...")

        # Faz o upload da imagem para a API
        arquivo_upload = genai.upload_file(path=caminho_foto)
        print(f"{caminho_foto.name} uploaded.")

        # --- OBJETIVO 1: Gerar JSON Estruturado (Usando o prompt pré-formatado) ---
        response_legenda_json = model_visao.generate_content([final_prompt, arquivo_upload])

        # Variáveis padrão em caso de falha no parse
        caption = "ERRO: Falha no parse"
        keywords_subjects = ""
        keywords_techniques = ""
        keywords_mood = ""
        category = ""
        context = ""

        try:
            # Limpar a resposta do modelo (às vezes vem com ```json ... ```)
            texto_limpo = response_legenda_json.text.strip().replace("```json", "").replace("```", "")
            dados_json = json.loads(texto_limpo)

            caption = dados_json.get('caption', 'ERRO: Missing caption')
            category = dados_json.get('category', '')
            context = dados_json.get('context', '')

            # Parsear o dicionário aninhado de keywords
            keywords_dict = dados_json.get('keywords', {})
            keywords_subjects = ", ".join(keywords_dict.get('subjects', []))
            keywords_techniques = ", ".join(keywords_dict.get('techniques', []))
            keywords_mood = ", ".join(keywords_dict.get('mood', []))

        except Exception as e:
            print(f"--- Aviso: Falha ao parsear JSON da legenda. Erro: {e}")
            print(f"--- Resposta Bruta: {response_legenda_json.text}")
            caption = response_legenda_json.text.strip() # Salva o texto bruto em caso de falha

        print(f"  Legenda: {caption}")
        print(f"  Categoria: {category}")
        print(f"  Contexto: {context}")
        print(f"  Keywords (Assuntos): {keywords_subjects}")
        print(f"  Keywords (Técnica): {keywords_techniques}")
        print(f"  Keywords (Sentimento): {keywords_mood}")

        # --- OBJETIVO 2 (Parte A): Gerar Embedding (DO TEXTO) ---

        # Criar um "super-texto" combinando todos os campos para um embedding rico
        texto_para_embedding = f"""
        Legenda: {caption}
        Categoria: {category}
        Contexto: {context}
        Assuntos: {keywords_subjects}
        Técnicas: {keywords_techniques}
        Sentimento: {keywords_mood}
        """

        response_embedding = genai.embed_content(
            model=MODELO_EMBEDDING,
            content=texto_para_embedding, # Usando o novo "super-texto"
            task_type="RETRIEVAL_DOCUMENT"
        )
        embedding = response_embedding['embedding']

        # Converter a lista de embedding em um texto JSON para salvar no CSV
        embedding_json = json.dumps(embedding)
        print(f"  Embedding do texto gerado.")


        # Guardar resultados
        resultados_finais.append({
            "arquivo": caminho_foto.name,
            "legenda": caption,
            "categoria": category,
            "contexto": context,
            "keywords_assuntos": keywords_subjects,
            "keywords_tecnica": keywords_techniques,
            "keywords_sentimento": keywords_mood,
            "embedding_json": embedding_json
        })

        # Pausa para evitar limite da API
        time.sleep(1)

    except Exception as e:
        print(f"!!! Erro ao processar {caminho_foto.name}: {e}")

    finally:
        # Bloco de limpeza
        if arquivo_upload:
            try:
                genai.delete_file(arquivo_upload.name)
            except Exception as e:
                print(f"--- Aviso: Falha ao limpar o arquivo {arquivo_upload.name}. {e}")

# 4. Salvar tudo em um arquivo CSV
df = pd.DataFrame(resultados_finais)
df.to_csv(ARQUIVO_DE_SAIDA, index=False)

print("\n--- SUCESSO! ---")
print(f"Processamento concluído. {len(df)} fotos únicas foram indexadas.")
print(f"Resultados salvos em: {ARQUIVO_DE_SAIDA}")
print(f"\nSeu CSV agora contém as colunas: {df.columns.to_list()}")

# BUSCA (Encontrar Fotos por Descrição Textual)

Bloco 3: Carrega seu índice na memória e prepara o modelo de IA. 
Digite a pasta onde está o arquivo index.csv quando solicitado. Você só precisa rodar uma vez por sessão.

In [ ]:
# Célula 3: Carregar o Índice de Fotos e o Modelo

# 1. Solicitar ao usuário a pasta onde está o index.csv
PASTA_DAS_FOTOS_BUSCA = input("Digite o caminho da pasta onde está o index.csv: ").strip()

# Remover aspas do caminho (simples ou duplas) que podem vir ao copiar do Windows
PASTA_DAS_FOTOS_BUSCA = PASTA_DAS_FOTOS_BUSCA.strip('"').strip("'")

# Definir o caminho completo do arquivo de índice
ARQUIVO_DE_INDICE = os.path.join(PASTA_DAS_FOTOS_BUSCA, "index.csv")

# 2. Carregar o modelo de embedding (para texto)
model_embedding = genai.GenerativeModel('models/embedding-001')

# 3. Carregar o índice de fotos do CSV
try:
    df_fotos = pd.read_csv(ARQUIVO_DE_INDICE)
    if df_fotos.empty:
        print(f"ERRO: O arquivo de índice '{ARQUIVO_DE_INDICE}' está vazio.")
    else:
        # Pré-processar os embeddings (converter JSON de volta para vetor)
        # Fazemos isso uma vez na carga para ser rápido
        df_fotos['embedding'] = df_fotos['embedding_json'].apply(lambda x: np.array(json.loads(x)))
        print(f"✓ Índice de {len(df_fotos)} fotos carregado e processado com sucesso.")

except FileNotFoundError:
    print(f"ERRO: Arquivo de índice '{ARQUIVO_DE_INDICE}' não encontrado.")
    print("Certifique-se de que você já executou a etapa de INDEXAÇÃO primeiro.")
except Exception as e:
    print(f"ERRO ao ler o arquivo de índice: {e}")

O Script de Busca (Seu Painel Principal). Digite a frase de busca quando solicitado e execute a célula para ver os resultados.

In [ ]:
# Célula 4: Script de Busca Semântica

# 1. Solicitar a frase de busca ao usuário
# Exemplos: "Momentos de alegria e diversão", "Paisagens naturais", "Pessoas sorrindo"
BUSCA_TEXTUAL = input("Digite a frase para buscar nas fotos: ").strip()

if not BUSCA_TEXTUAL:
    print("ERRO: Você precisa digitar uma frase para buscar!")
elif 'df_fotos' not in locals() or df_fotos.empty:
    print("ERRO: O índice de fotos não foi carregado. Execute a Célula 3 primeiro.")
else:
    print(f"\n🔍 Buscando por: '{BUSCA_TEXTUAL}'")

    # 2. Gerar o embedding para a busca textual (a "query")
    response_query = genai.embed_content(
        model='models/embedding-001',
        content=BUSCA_TEXTUAL,
        task_type="RETRIEVAL_QUERY" # Importante: "RETRIEVAL_QUERY" para busca
    )
    query_embedding = np.array(response_query['embedding'])

    # 3. Calcular a similaridade (Produto Escalar)
    #    (Compara o vetor da busca com CADA vetor de texto da foto)
    df_fotos['similaridade'] = df_fotos['embedding'].apply(
        lambda texto_emb: np.dot(query_embedding, texto_emb)
    )

    # 4. Ordenar e mostrar os melhores resultados
    df_resultados = df_fotos.sort_values(by='similaridade', ascending=False)

    print("\n--- 🏆 MELHORES FOTOS ENCONTRADAS 🏆 ---")
    print(f"Busca: '{BUSCA_TEXTUAL}'\n")

    # 5. Mostrar as 5 fotos mais relevantes de forma legível
    for index, row in df_resultados.head(5).iterrows():
        print(f"--- Posição #{index+1} (Similaridade: {row['similaridade']:.4f}) ---")
        print(f"Arquivo: {row['arquivo']}")
        print(f"Legenda: {row['legenda']}")
        print(f"Contexto: {row['contexto']}")
        print(f"Keywords (Assuntos): {row['keywords_assuntos']}")
        print("\n")